# Redis in Python

This notebook will showcase how to interact with a Redis db via the Python library. 

Useful resources:
<ul>
    <li><a href="https://redis-py.readthedocs.io/en/stable/" target="_blank">Redis Python documentation</a></li>
    <li><a href="https://redis.io/docs/getting-started/" target="_blank">Base Redis documentation</a></li>
    <li><a href="https://www.ibm.com/docs/en/watson-studio-local/1.2.3?topic=notebooks-markdown-jupyter-cheatsheet" target="_blank">Markdown in jupyter cheatsheet</a></li>
</ul>

## Getting started:

In this section we will get started by connecting to our redis db, take a look at exisiting data, and checkout a few datatypes.

Start by importing redis:

In [4]:
import redis

Next we connect to our Redis db: 

In [14]:
redis_client = redis.Redis(host='localhost', port=6379, db=0)

<div class="alert alert-block alert-info">
    After writing this tutorial, I learned that you can add `decode_responses=True` when you initialize the redis client to automatically decode bytes to strings. Adding this allows you to skip the decoding step below.
</div>

To start, let's take a look at keys that I have already added to the Redis db:

In [15]:
redis_client.keys()

[b'collection-info:61',
 b'collection-info:14',
 b'collection:14',
 b'collection:61',
 b'collection:67',
 b'collection-info:12',
 b'collection:12',
 b'collection-info:67',
 b'collection:62',
 b'collection-info:62']

The first thing that we notice is that the keys are returned as byte strings. Although this doesn't matter now while we are simply exploring the data, it is important to note that we must convert these to strings if we plan on using them in python. We can quickly convert these to normal strings:

In [16]:
raw_redis_keys = redis_client.keys()
redis_keys = [x.decode('utf-8') for x in raw_redis_keys]
redis_keys

['collection-info:61',
 'collection-info:14',
 'collection:14',
 'collection:61',
 'collection:67',
 'collection-info:12',
 'collection:12',
 'collection-info:67',
 'collection:62',
 'collection-info:62']

Now that we know that certain keys exist, let's check the data types so that we can properly query the data:

In [17]:
redis_client.type('collection:14')

b'zset'

Here we see that `collection:<id>` is a sorted set. This is also how geographic data is stored.

In [18]:
redis_client.type('collection-info:14')

b'hash'

`collection-info:<id>` is stored as a hash.

<div class="alert alert-block alert-success">
    Keys can contain different datatypes, each of which requires a specific call to access.
</div

## Accessing data stored in different datatypes 

### Sorted Sets (zset):

In [21]:
coll_resource_list = redis_client.zrange("collection:62", 0, -1)

In [22]:
# redis will return data as bytes so we need to transform this
# we extract the resource id str from the key name
cr_list = [x.decode('utf-8').split(':')[1] for x in coll_resource_list]

In [23]:
cr_list

['59',
 '314',
 '315',
 '377',
 '379',
 '387',
 '395',
 '398',
 '896',
 '1037',
 '1856',
 '1867',
 '1875',
 '2426',
 '2653',
 '3104',
 '3167',
 '3179',
 '3259',
 '5521',
 '7950',
 '8257',
 '8732',
 '8742',
 '9213',
 '10167',
 '10332',
 '10390',
 '10404',
 '10634',
 '10684',
 '10752',
 '10881',
 '11139',
 '11149',
 '11151',
 '13094',
 '19637',
 '19712',
 '19713',
 '19776',
 '46642',
 '47598',
 '49618',
 '49619',
 '49620',
 '49621',
 '49622',
 '49623',
 '49624',
 '49625',
 '49626',
 '49627',
 '49628',
 '49629',
 '49630',
 '49631',
 '49632',
 '49633',
 '49634',
 '49635',
 '49636',
 '49637',
 '49638',
 '49639',
 '49640',
 '49641',
 '49642',
 '49643',
 '49644',
 '49646',
 '49647',
 '49648',
 '49649',
 '49650',
 '49651',
 '49652',
 '49653',
 '49654',
 '49655',
 '49656',
 '49657',
 '49658',
 '49659',
 '49660',
 '49661',
 '49662',
 '49663',
 '49664',
 '49665',
 '49666',
 '49667',
 '49668',
 '49669',
 '49670',
 '49671',
 '49672',
 '49673',
 '49674',
 '49675',
 '49676',
 '49677',
 '49678',
 '496

In [27]:
def get_collection_title(collection_id):
    collection_identifier = "collection-info:{0}".format(collection_id)
    # here is the redis magic
    coll_title = redis_client.hget(collection_identifier, "title")
    if coll_title:
        coll_title = coll_title.decode('utf-8')
    else:
        coll_title = ''
    return coll_title


In [28]:
get_collection_title(62)

'Animal Telemetry Project'

In [29]:
get_collection_title(188)

''

In [34]:
test_pattern = 'collection:*'

In [35]:
for key in redis_client.scan_iter(match=test_pattern):
    print(key)

b'collection:14'
b'collection:61'
b'collection:62'
b'collection:67'
b'collection:12'
